In [3]:
import pdfplumber

def extract_text(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
    return full_text


In [4]:
import re

def normalize(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text


In [5]:
patterns = {
    "effective_date": r"Efective Date.*?(\d{4}\.\d{2}\.\d{2})",
    "revision_no": r"Revision No.*?:\s*([A-Z])",
    "analysis_date": r"Analysis Date.*?(\d{4}\.\d{2}\.\d{2})",
    "sampler": r"Sampler.*?\)\s*([A-Za-z ]+)",
    "analyst": r"Analyst Name.*?\)\s*([A-Za-z ]+)",
}
patterns.update({
    "total_moisture_ar": r"Total Moisture.*?As Received Basis.*?%\s*([\d\.]+)",
    "inherent_moisture_ad": r"Inherent moisture.*?Air Dried Basis.*?%\s*([\d\.]+)",
    "ash_ar": r"Ash Contant.*?As Received Basis.*?%\s*([\d\.]+)",
    "ash_ad": r"Ash Contant.*?Air Dried Basis.*?%\s*([\d\.]+)",
    "vm_ar": r"Volatile Matter.*?As Received Basis.*?%\s*([\d\.]+)",
    "vm_ad": r"Volatile Matter.*?Air Dried Basis.*?%\s*([\d\.]+)",
    "sulphur_ar": r"Total sulphur.*?As Received Basis.*?%\s*([\d\.]+)",
    "sulphur_ad": r"Total sulphur.*?Air Dried Basis.*?%\s*([\d\.]+)",
    "fixed_carbon_ar": r"Fixed carbon.*?As Received Basis.*?%\s*([\d\.]+)",
    "fixed_carbon_ad": r"Fixed carbon.*?Air Dried Basis.*?%\s*([\d\.]+)",
})
patterns.update({
    "gcv_ar": r"Gross Calorific Value.*?As Received Basis.*?Cal/g\s*([\d\.]+)",
    "gcv_ad": r"Gross Calorific Value.*?Air Dried Basis.*?Cal/g\s*([\d\.]+)",
    "ncv_ar": r"Net Calorific Value.*?As Received Basis.*?Cal/g\s*([\d\.]+)",
    "ncv_ad": r"Net Calorific Value.*?Air Dried Basis.*?Cal/g\s*([\d\.]+)",
})
patterns.update({
    "dt": r"Deformatio Temperature.*?°C\s*([\d\.]+)",
    "st": r"Softening Temperature.*?°C\s*([\d\.]+)",
    "ht": r"Hemisphere Temperature.*?°C\s*([\d\.]+)",
    "ft": r"Flow Temperature.*?°C\s*([\d\.]+)",
})
def extract_all(text, patterns):
    results = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        results[key] = match.group(1) if match else None
    return results


In [6]:
COLUMNS = ['effective_date',
 'analysis_date',
 'total_moisture_ar',
 'inherent_moisture_ad',
 'ash_ar',
 'ash_ad',
 'sulphur_ar',
 'sulphur_ad',
 'fixed_carbon_ar',
 'fixed_carbon_ad',
 'gcv_ar',
 'gcv_ad',
 'dt',
 'st',
]

For As Fired Coal Data

In [ ]:
import os
from datetime import datetime as dt

def get_as_fired_pdfs(folder_path):
    return [
        f for f in os.listdir(folder_path)
        if f.lower().endswith(".pdf")
        and "as fired" in f.lower()
    ]

pdf_files = get_as_fired_pdfs("pdfs/")
for pdf_file in pdf_files:
    raw_text = extract_text('pdfs//' + pdf_file)
    clean_text = normalize(raw_text)
    data = extract_all(clean_text, patterns)
    row = [data.get(col, "") for col in COLUMNS]

    # Converting Date String into Datetime object
    row[0] = dt.strptime(row[0], '%Y.%m.%d')
    row[1] = dt.strptime(row[1], '%Y.%m.%d')

    from openpyxl import load_workbook

    wb = load_workbook("coal_data_as_fired.xlsx")
    ws = wb.active

    ws.append(row)
    wb.save("coal_data_as_fired.xlsx")